Az osztály(oka)t mentsd a __src/linear_regression__ modul __LinearRegressions.py__ fájljába
Használható modulok: _pathlib, pandas, typing, str, numpy, scipy.stats t_ és _norm_, valamint a _scipy.optimize_ _minimize_ osztályai 

In [ ]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

In [ ]:
datalib = Path.cwd().parent.joinpath('data')

### Adatelőkészítés (0 pont)
1., Olvasd be a data mappa __sp500.parquet__ nevű fájlját egy DataFrame-be. A betöltéshez használt engine paramétere legyen <u>fastparquet</u>
2., Olvasd be az __ff_factors.parquet__ fájlt egy DataFrame-be. A betöltéshez használt engine paramétere legyen <u>fastparquet</u>
3., Kapcsold össze a két DataFrame-t egy új DataFrame-be. Az összekapcsolás módja, hogy a hozam adatokra balról kapcsoljuk rá a factor adatokat a __'Date' elsődleges kulcs__ alapján.
4., Készíts egy új __'Excess Return'__ nevű oszlopot, ami a havi hozamok és a kockázat mentes hozam (RF) különbsége
5., <u>Rendezd sorba dátum szerint az adatokat</u>, majd generálj egy új oszlopot (__'ex_ret_1'__), amely minden ticker ('Symbol') esetén 1-el eltolja az Excess Return értékeit olyan módon, hogy minden sorban szerepeljen a <u>következő időszaki Excess Return</u> érték. 
6., A meglévő adathalmazt írd felül olyan módon, hogy egyszer <u>törlöd az össze olyan sort</u>, amely az __'ex_ret_1' oszlopban hiányos__, majd ezt követően, törlöd az összes olyan sort, ami a __'HML' oszlopban hiányos__.


3. feladat segítség
![Joined data](../resources/weekly6/joined_data.jpg)

5. feladat segítség
![new column](../resources/weekly6/ex_ret_1.jpg)

In [4]:
from pathlib import Path
import pandas as pd

file_path = Path("/Users/molnartamas07/Documents/ECOPY_23241/data/sp500.parquet")

df = pd.read_parquet(file_path, engine='fastparquet')

file_path = Path("/Users/molnartamas07/Documents/ECOPY_23241/data/ff_factors.parquet")

df_ff_factors = pd.read_parquet(file_path, engine='fastparquet')

merged_df = pd.merge(df_sp500, df_ff_factors, on='Date', how='left')

merged_df['Excess Return'] = merged_df['Return'] - merged_df['RF']

merged_df.sort_values(by='Date', inplace=True)

merged_df['ex_ret_1'] = merged_df.groupby('Symbol')['Excess Return'].shift(-1)

merged_df = merged_df.dropna(subset=['ex_ret_1'])

merged_df = merged_df.dropna(subset=['HML'])

amazon_df = merged_df[merged_df['Symbol'] == 'AMZN']

amazon_df = amazon_df.drop(columns=['Symbol'])


ImportError: Unable to import required dependencies:
numpy: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.11 from "/Users/molnartamas07/Documents/ECOPY_23241/venv/bin/python"
  * The NumPy version is: "1.25.2"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: No module named 'numpy.core._multiarray_umath'


### Modell összeállítás (7 pont)

A meglévő adatokból válaszd ki a __Amazon részvényhez tartozó sorokat (AMZN)__ és töröld a tickereket tartalmazó oszlopot. <u>Amennyiben működött, önellenőrzésre használhatod a legutóbbi zárthelyin írt osztályt is.</u>

7., Készíts egy új __LinearRegressionML__ elnevezésű osztályt. Definiáld benne a __\_\_init\_\___ nevű függvényt, amely bemenetként 2 DataFrame-t kap amelyeket ments le a left_hand_side és right_hand_side elnevezésű változókba. Az egyik DataFrame fogja tartalmazni a következő hónap többlet hozamait (__left_hand_side__), a másik a piaci hozamokat (Mkt-RF), az SMB és a HML értékeket (__right_hand_side__).

8., Egésztsd ki az osztályt egy __fit__ metódussal, ami ML elvű becslést hajt végre. Figyelj oda, hogy a regresszió futtatása során konstans (alfa / béta_0) is szerepeljen a predictor változók között, amely az első változó legyen. __A feladatot numerikus optimalizálással old meg, analítikus megoldásra nem ját pont__

9., Egészítsd ki az osztályt egy __get_params__ metódussal, ami visszaadja a becsült modell béta paramétereinek értékeit. A visszakapott pandas Series típusú adatban az oszlop neve legyen __Beta coefficients__. 

10., Egészítsd ki az osztályt egy __get_pvalues__ metódussal, ami visszaadja a becsült modell paraméterekhez tartozó p értékeket. A visszakapott pandas Series típusú adatban az oszlop neve legyen: __P-values for the corresponding coefficients__. A p értéket t-statisztika alapján számold ki. A p-érték kiszámításánál figyelj, hogy  alkalmazd a <u>min(value, 1-value) * 2</u> képletet.

11., Egészítse ki az osztályt egy __get_model_goodness_values__ metódussal, ami visszadja a centrált és a módosított R-négyzet értékeket. A visszatérési típus string legyen, a visszaadandó szöveg: __Centered R-squared: crs, Adjusted R-squared: ars__, ahol crs és ars helyére 3 tizedesjegyre kerekítve (__:.3f__) add meg a hozzájuk tartozó értékeket. <u>Ha a regresszorok számába eredetileg beleszámítottad a konstanst is, akkor a módosított R-négyzet számítás nevezőjében nincs szükség a __-1__-es tagra</u>.

#### Kiegészítés:
- Mivel a célparaméterek kicsik, a minimalizálás kiinduló értékeit érdemes egységesen, minden paraméter számára __0.1__-re állítani.
- Javasolt a __L-BFGS-B__ optimalizáló alkalmazása
- Ne felejtsd el, hogy az MLE __variancia becslése torzított__. A kapott eredményeket ennek megfelelően korrigáld.

In [2]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

class LinearRegressionML:
    def __init__(self, left_hand_side, right_hand_side):
        if not isinstance(left_hand_side, pd.DataFrame) or not isinstance(right_hand_side, pd.DataFrame):
            raise ValueError("Both inputs should be DataFrames.")

        self.left_hand_side = left_hand_side
        self.right_hand_side = right_hand_side
        self.beta = None

    def fit(self):
        n_predictors = self.right_hand_side.shape[1]
        initial_params = np.full(n_predictors + 2, 0.1)  # Setting initial values to 0.1 for all parameters
        result = minimize(self._negative_log_likelihood, initial_params, method='L-BFGS-B')

        if result.success:
            self.beta = result.x[1:]
            self._correct_mle_variance(result.hess_inv)
        else:
            raise ValueError("Optimization did not converge. Please check your input data.")

    def _negative_log_likelihood(self, params):
        alpha = params[0]
        beta = params[1:]
        y = self.left_hand_side.values
        X = np.column_stack((np.ones(len(self.left_hand_side)), self.right_hand_side.values))
        epsilon = y - X @ beta - alpha
        neg_log_likelihood = 0.5 * np.sum(epsilon ** 2)
        return neg_log_likelihood

    def _correct_mle_variance(self, hessian_inv):
        n, k = self.right_hand_side.shape[0], self.right_hand_side.shape[1]
        sigma_squared = np.sum((self.left_hand_side - self.right_hand_side @ self.beta) ** 2) / (n - k)
        cov_matrix = hessian_inv * sigma_squared
        std_errors = np.sqrt(np.diag(cov_matrix))
        self.std_errors = std_errors

    def get_params(self):
        if not hasattr(self, 'beta'):
            raise ValueError("Model parameters have not been estimated. Please call fit() first.")
        params_with_alpha = [np.nan] + list(self.beta)
        return pd.Series(params_with_alpha, name='Beta coefficients')

    def get_model_goodness_values(self):
        if not hasattr(self, 'beta'):
            raise ValueError("Model parameters have not been estimated. Please call fit() first.")

        n, k = self.right_hand_side.shape[0], self.right_hand_side.shape[1]

        y_mean = self.left_hand_side.mean().values
        y_hat = self.right_hand_side @ self.beta
        rss = np.sum((self.left_hand_side - y_hat) ** 2)
        tss = np.sum((self.left_hand_side - y_mean) ** 2)
        crs = 1 - rss / tss

        if k > 1:
            ars = 1 - (rss / (n - k)) / (tss / (n - 1))
        else:
            ars = crs

        return f"Centered R-squared: {crs:.3f}, Adjusted R-squared: {ars:.3f}"

ImportError: Unable to import required dependencies:
numpy: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.11 from "/Users/molnartamas07/Documents/ECOPY_23241/venv/bin/python"
  * The NumPy version is: "1.25.2"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: No module named 'numpy.core._multiarray_umath'
